# OPENAI PRODUCT PRICER

In [1]:
import os
import random 
from huggingface_hub import login
import re
import json
import random
from dotenv import load_dotenv
from openai import OpenAI
import pickle


In [2]:
from testing import Tester

In [3]:
load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
os.environ['HF_TOKEN'] = os.getenv('HF_TOKEN', 'your-key-if-not-using-env')

In [4]:
hf_token = os.environ["HF_TOKEN"]
login(hf_token, add_to_git_credential=True)

openai = OpenAI()

%matplotlib inline

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [5]:
with open('train.pkl', 'rb') as file:
    train = pickle.load(file)

with open('test.pkl', 'rb') as file:
    test = pickle.load(file)

In [6]:
print(train[0].prompt)
print(test[0].prompt)

How much does this cost to the nearest dollar?

Delphi FG0166 Fuel Pump Module
Delphi brings 80 years of OE Heritage into each Delphi pump, ensuring quality and fitment for each Delphi part. Part is validated, tested and matched to the right vehicle application Delphi brings 80 years of OE Heritage into each Delphi assembly, ensuring quality and fitment for each Delphi part Always be sure to check and clean fuel tank to avoid unnecessary returns Rigorous OE-testing ensures the pump can withstand extreme temperatures Brand Delphi, Fit Type Vehicle Specific Fit, Dimensions LxWxH 19.7 x 7.7 x 5.1 inches, Weight 2.2 Pounds, Auto Part Position Unknown, Operation Mode Mechanical, Manufacturer Delphi, Model FUEL PUMP, Dimensions 19.7

Price is $227.00
How much does this cost to the nearest dollar?

OEM AC Compressor w/A/C Repair Kit For Ford F150 F-150 V8 & Lincoln Mark LT 2007 2008 - BuyAutoParts NEW
As one of the world's largest automotive parts suppliers, our parts are trusted every day by

In [7]:
class Tester:

    def __init__(self, predictor, title=None, data=test, size=250):
        self.predictor = predictor
        self.data = data
        self.title = title or predictor.__name__.replace("_", " ").title()
        self.size = size
        self.guesses = []
        self.truths = []
        self.errors = []
        self.sles = []
        self.colors = []

    def color_for(self, error, truth):
        if error<40 or error/truth < 0.2:
            return "green"
        elif error<80 or error/truth < 0.4:
            return "orange"
        else:
            return "red"
    
    def run_datapoint(self, i):
        datapoint = self.data[i]
        guess = self.predictor(datapoint)
        truth = datapoint.price
        error = abs(guess - truth)
        log_error = math.log(truth+1) - math.log(guess+1)
        sle = log_error ** 2
        color = self.color_for(error, truth)
        title = datapoint.title if len(datapoint.title) <= 40 else datapoint.title[:40]+"..."
        self.guesses.append(guess)
        self.truths.append(truth)
        self.errors.append(error)
        self.sles.append(sle)
        self.colors.append(color)
        print(f"{COLOR_MAP[color]}{i+1}: Guess: ${guess:,.2f} Truth: ${truth:,.2f} Error: ${error:,.2f} SLE: {sle:,.2f} Item: {title}{RESET}")

    def chart(self, title):
        max_error = max(self.errors)
        plt.figure(figsize=(12, 8))
        max_val = max(max(self.truths), max(self.guesses))
        plt.plot([0, max_val], [0, max_val], color='deepskyblue', lw=2, alpha=0.6)
        plt.scatter(self.truths, self.guesses, s=3, c=self.colors)
        plt.xlabel('Ground Truth')
        plt.ylabel('Model Estimate')
        plt.xlim(0, max_val)
        plt.ylim(0, max_val)
        plt.title(title)
        plt.show()

    def report(self):
        average_error = sum(self.errors) / self.size
        rmsle = math.sqrt(sum(self.sles) / self.size)
        hits = sum(1 for color in self.colors if color=="green")
        title = f"{self.title} Error=${average_error:,.2f} RMSLE={rmsle:,.2f} Hits={hits/self.size*100:.1f}%"
        self.chart(title)

    def run(self):
        self.error = 0
        for i in range(self.size):
            self.run_datapoint(i)
        self.report()

    @classmethod
    def test(cls, function):
        cls(function).run()

In [8]:
# OpenAI recommends fine-tuning with populations of 50-100 examples

fine_tune_train = train[:200]
fine_tune_validation = train[200:250]

In [9]:
# Prepare data for fine tuning in JSONL format and upload to openai

def messages_for(item):
    system_message = "You estimate prices of items. Reply only with the price, no explanation"
    user_prompt = item.test_prompt().replace(" to the nearest dollar","").replace("\n\nPrice is $","")
    return [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_prompt},
        {"role": "assistant", "content": f"Price is ${item.price:.2f}"}
    ]

messages_for(fine_tune_train[0])

[{'role': 'system',
  'content': 'You estimate prices of items. Reply only with the price, no explanation'},
 {'role': 'user',
  'content': 'How much does this cost?\n\nDelphi FG0166 Fuel Pump Module\nDelphi brings 80 years of OE Heritage into each Delphi pump, ensuring quality and fitment for each Delphi part. Part is validated, tested and matched to the right vehicle application Delphi brings 80 years of OE Heritage into each Delphi assembly, ensuring quality and fitment for each Delphi part Always be sure to check and clean fuel tank to avoid unnecessary returns Rigorous OE-testing ensures the pump can withstand extreme temperatures Brand Delphi, Fit Type Vehicle Specific Fit, Dimensions LxWxH 19.7 x 7.7 x 5.1 inches, Weight 2.2 Pounds, Auto Part Position Unknown, Operation Mode Mechanical, Manufacturer Delphi, Model FUEL PUMP, Dimensions 19.7'},
 {'role': 'assistant', 'content': 'Price is $226.95'}]

In [10]:
def to_jsonl(items):
    res = ""
    for item in items:
        message = messages_for(item)
        msg_str = json.dumps(message)
        res += '{"messages": ' + msg_str +'}\n'

    return res

print(to_jsonl(train[:5]))

{"messages": [{"role": "system", "content": "You estimate prices of items. Reply only with the price, no explanation"}, {"role": "user", "content": "How much does this cost?\n\nDelphi FG0166 Fuel Pump Module\nDelphi brings 80 years of OE Heritage into each Delphi pump, ensuring quality and fitment for each Delphi part. Part is validated, tested and matched to the right vehicle application Delphi brings 80 years of OE Heritage into each Delphi assembly, ensuring quality and fitment for each Delphi part Always be sure to check and clean fuel tank to avoid unnecessary returns Rigorous OE-testing ensures the pump can withstand extreme temperatures Brand Delphi, Fit Type Vehicle Specific Fit, Dimensions LxWxH 19.7 x 7.7 x 5.1 inches, Weight 2.2 Pounds, Auto Part Position Unknown, Operation Mode Mechanical, Manufacturer Delphi, Model FUEL PUMP, Dimensions 19.7"}, {"role": "assistant", "content": "Price is $226.95"}]}
{"messages": [{"role": "system", "content": "You estimate prices of items. 

In [11]:
def write_to_jsonl_file(items, filename):
    with open(filename, "w") as f:
        f.write(to_jsonl(items))

In [12]:
write_to_jsonl_file(fine_tune_train, "fine_tune_train.jsonl")
write_to_jsonl_file(fine_tune_validation, "fine_tune_validation.jsonl")

In [13]:
with open("fine_tune_train.jsonl", "rb") as f:
    train_file = openai.files.create(file=f, purpose="fine-tune")

with open("fine_tune_validation.jsonl", "rb") as f:
    validation_file = openai.files.create(file=f, purpose="fine-tune")

In [14]:
wandb_integrations = {'type':'wandb', 'wandb':{'project':'apenai-product-pricer'}}

In [ ]:
# more times you run this, more the jobs created, so be careful while running this

openai.fine_tuning.jobs.create(
    training_file=train_file.id,
    validation_file=validation_file.id,
    model="gpt-4o-mini-2024-07-18",
    seed=42,
    hyperparameters={'n_epochs':1},
    integrations= [wandb_integrations],
    suffix = "pricer"
)

FineTuningJob(id='ftjob-zxAHbTnY6r606wzT11fzBCRZ', created_at=1741076892, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(batch_size='auto', learning_rate_multiplier='auto', n_epochs=1), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-V6i4MUSv3KwU5OH0TUGZiVjl', result_files=[], seed=42, status='validating_files', trained_tokens=None, training_file='file-DTLvicDdY2oU7HFFNBFuXP', validation_file='file-MHZiLDwRpZCuMX6JcTvt5Z', estimated_finish=None, integrations=[FineTuningJobWandbIntegrationObject(type='wandb', wandb=FineTuningJobWandbIntegration(project='apenai-product-pricer', entity=None, name=None, tags=None, run_id='ftjob-zxAHbTnY6r606wzT11fzBCRZ'))], method=Method(dpo=None, supervised=MethodSupervised(hyperparameters=MethodSupervisedHyperparameters(batch_size='auto', learning_rate_multiplier='auto', n_epochs=1)), type='supervised'), user_provided_suffix='pricer', metadata=None)

In [22]:
openai.fine_tuning.jobs.list(limit=1)
job_id = openai.fine_tuning.jobs.list(limit=1).data[0].id
job_id

'ftjob-vuuq7AgmTJbhdA9nZ3rIvf7S'

In [23]:
openai.fine_tuning.jobs.retrieve('ftjob-vuuq7AgmTJbhdA9nZ3rIvf7S')

FineTuningJob(id='ftjob-vuuq7AgmTJbhdA9nZ3rIvf7S', created_at=1741076709, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(batch_size=1, learning_rate_multiplier=1.8, n_epochs=1), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-V6i4MUSv3KwU5OH0TUGZiVjl', result_files=[], seed=32, status='running', trained_tokens=None, training_file='file-DTLvicDdY2oU7HFFNBFuXP', validation_file='file-MHZiLDwRpZCuMX6JcTvt5Z', estimated_finish=None, integrations=[FineTuningJobWandbIntegrationObject(type='wandb', wandb=FineTuningJobWandbIntegration(project='apenai-product-pricer', entity=None, name=None, tags=None, run_id='ftjob-vuuq7AgmTJbhdA9nZ3rIvf7S'))], method=Method(dpo=None, supervised=MethodSupervised(hyperparameters=MethodSupervisedHyperparameters(batch_size=1, learning_rate_multiplier=1.8, n_epochs=1)), type='supervised'), user_provided_suffix='pricer', metadata=None)

In [25]:
openai.fine_tuning.jobs.list_events(fine_tuning_job_id=job_id, limit=10).data

[FineTuningJobEvent(id='ftevent-qcnWwK7uYgaIqJgZ2YkP31uv', created_at=1741076840, level='info', message='Fine-tuning job started', object='fine_tuning.job.event', data=None, type='message'),
 FineTuningJobEvent(id='ftevent-a6bUeKzTCXkeF9R2bm5UCuvt', created_at=1741076839, level='info', message='Files validated, moving job to queued state', object='fine_tuning.job.event', data={}, type='message'),
 FineTuningJobEvent(id='ftevent-oaASORtYX0NuZFxj0iZo0uS1', created_at=1741076710, level='info', message='Validating training file: file-DTLvicDdY2oU7HFFNBFuXP and validation file: file-MHZiLDwRpZCuMX6JcTvt5Z', object='fine_tuning.job.event', data={}, type='message'),
 FineTuningJobEvent(id='ftevent-bYaX3QmD8JmnDiWCP2e8fwav', created_at=1741076710, level='info', message='Created fine-tuning job: ftjob-vuuq7AgmTJbhdA9nZ3rIvf7S', object='fine_tuning.job.event', data={}, type='message')]

In [28]:
fined_tuned_model = "ft:gpt-4o-mini-2024-07-18:sidiboiorg:pricer:B7I1ke2E" # get this by going to openai api page and clicking fine-tuning in dashboard section

In [29]:
def messages_for(item):
    system_message = "You estimate prices of items. Reply only with the price, no explanation"
    user_prompt = item.test_prompt().replace(" to the nearest dollar","").replace("\n\nPrice is $","")
    return [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_prompt},
        {"role": "assistant", "content": "Price is $"}
    ]

In [30]:
# A utility function to extract the price from a string

def get_price(s):
    s = s.replace('$','').replace(',','')
    match = re.search(r"[-+]?\d*\.\d+|\d+", s)
    return float(match.group()) if match else 0

In [31]:
def get_price_fine_tuned(item):
    response = openai.chat.completions.create(
        model=fined_tuned_model,
        max_tokens = 7,
        seed = 42,
        messages=messages_for(item)
    )
    reply = response.choices[0].message.content
    return get_price(reply)

In [ ]:
print(test[0].price)
print(get_price_fine_tuned(test[0]))

374.41
490.0


In [34]:
print(test[1].price)
print(get_price_fine_tuned(test[1]))

225.11
194.99
